In [7]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[ 0.0652, -0.1084,  0.1204, -0.0322, -0.2146, -0.0046,  0.1049,  0.0402,
          0.0465, -0.0093],
        [ 0.1525, -0.1459, -0.0632,  0.0572, -0.1844, -0.0497, -0.0406,  0.1483,
          0.0375, -0.0305]], grad_fn=<AddmmBackward0>)

In [8]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [9]:
net = MLP()
net(X)

tensor([[ 0.1626, -0.0356,  0.1331, -0.1241,  0.0494,  0.0161, -0.0850,  0.0743,
         -0.0521,  0.0167],
        [ 0.1331,  0.0108,  0.2130, -0.0633,  0.1747, -0.0509, -0.0029,  0.0788,
         -0.0532,  0.0267]], grad_fn=<AddmmBackward0>)

In [10]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            self._modules[block] = block

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.0301,  0.1627, -0.2045, -0.0057,  0.0273,  0.0475,  0.1162, -0.0519,
         -0.0919,  0.1747],
        [-0.0929,  0.1639, -0.2541, -0.0171,  0.0063,  0.0918,  0.1062, -0.1741,
         -0.1879,  0.2935]], grad_fn=<AddmmBackward0>)

In [11]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(0.0509, grad_fn=<SumBackward0>)

In [12]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.5177, grad_fn=<SumBackward0>)